In [3]:
pip install qutip

Note: you may need to restart the kernel to use updated packages.


In [70]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from qutip.qip.operations import *
np.set_printoptions(precision = 3)

In [36]:
# spin operators
X = sigmax()
Y = sigmay()
Z = sigmaz()
I = qeye(2)

II = tensor(I, I)

IX = tensor(X, I)
IY = tensor(Y, I)
IZ = tensor(Z, I)

SX = tensor(I, X)
SY = tensor(I, Y)
SZ = tensor(I, Z)

IZSZ = tensor(Z, Z) # J coupling

In [37]:
R = lambda state, angle: ((state * -1j * angle) / 2).expm()

Ix = lambda angle: R(IX, angle)
Sx = lambda angle: R(SX, angle)
IxSx = lambda angle: R(IX, angle) + R(SX, angle)

Iy = lambda angle: R(IY, angle)
Sy = lambda angle: R(SY, angle)
IySy = lambda angle: R(SX, angle) + R(SY, angle)

# we cannot pulse on Z in NMR in the same way
# so we do a composite of X and Y pulse
Iz = lambda angle: Ix(np.pi/2) * Iy(angle) * Ix(-np.pi/2)
Sz = lambda angle: Sx(np.pi/2) * Sy(angle) * Sx(-np.pi/2)
IzSz = lambda angle: Iz(angle) * Sz(angle)

##### Pulse Sequence Phases (3rd Param)
+x = 0

+y = 1

-x = 2

-y = 3

### Bit Flip

Pulse Sequence for 1H: 
`pulse(1, a90H, 0, d180H)`\
Pulse Sequence for 13C:
`pulse(2, a90C, 0, d180C)`\
Pulse Sequence for both:
`pulse(1, a90HC, 0, freq1H, 2, a90C, 0, freq13C, d180C)`

In [56]:
def bitflip_operator(qubit):
    if qubit == "1H":
        return Ix(-np.pi)
    elif qubit == "13C":
        return Sx(-np.pi)
    elif qubit == "both":
        return IxSx(-np.pi)
    else: 
        print(f"Invalid qubit for bitflip. Do nothing.")
        return II

In [66]:
print(np.array(bitflip_operator('1H')).real)
print(np.array(bitflip_operator('13C')).real)
print(np.array(bitflip_operator('both')).real)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Hadamard

Pulse Sequence for 1H:

`pulse(1, a90H, 0, d180H)`\
`delay(0.25)`\
`pulse(1, a90H, 1, d90H)`

Pulse Sequence for 13C:

`pulse(2, a90C, 0, d180C)`\
`delay(0.25)`\
`pulse(2, a90C, 1, d180C)`


Pulse Sequence for both:

`pulse(1, a90HC, 0, freq1H, 2, a90C, 0, freq13, d180C)`\
`delay(0.25)`\
`pulse(1, a90HC, 1, freq1H, 2, a90C, 1, freq13, d90C)`

In [57]:
def hadamard_operator(qubit):
    if qubit == "1H":
        return Ix(np.pi) * Iy(np.pi/2)
    
    elif qubit == "13C":
        return Sx(np.pi) * Sy(np.pi/2)

    elif qubit == "both":
        return IxSx(np.pi) * IySy(np.pi/2)
    
    else: 
        print(f"Invalid qubit for Hadamard. Do nothing.")
        return II

In [72]:
print(np.sqrt(2) * np.array(hadamard_operator('1H')).real)
print(np.sqrt(2) * np.array(hadamard_operator('13C')).real)
print(np.sqrt(2) * np.array(hadamard_operator('both')).real)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[-1.  0.  0. -1.]
 [ 0. -1. -1.  0.]
 [ 0. -1. -1.  0.]
 [-1.  0.  0. -1.]]


### CZ

Composite Iz pulse on NMR:

`pulse(1, a90H, 0, d90H)`\
`delay(0.25)`\
`pulse(1, a90H, 1, d90H)`\
`delay(0.25)`\
`pulse(1, d90H, 2, d90H)`\
`delay(0.25)`

Composite Sz pulse on NMR:\
`pulse(2, a90C, 0, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 1, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 2, d90C)`

In [58]:
def CZ_operator():
    Iz = Ix(np.pi/2) * Iy(np.pi/2) * Ix(-np.pi/2)
    Sz = Sx(np.pi/2) * Sy(np.pi/2) * Sx(-np.pi/2)
    return  Iz * Sz * IzSz(-np.pi) # what we can implement on NMR

In [74]:
print(np.array(CZ_operator()))

[[0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]]


#### CNOT

Pulse Sequence for Approximate CNOT:

`pulse(2, a90C, 0, d90C)` \
`delay(dEvolution)` \
`pulse(2, a90C, 1, d90C)`

Pulse Sequence for Actual CNOT:

`pulse(2, a90C, 1, d45C)`\
`delay(0.25)`\
`pulse(2, a90C, 0, d180C)`\
`delay(0.25)`\
`pulse(2, a90C, 3, d45C)`\
`delay(0.25)`

`pulse(1, a90H, 2, d90H)`\
`delay(0.25)`\
`pulse(1, a90H, 1, d90H)`\
`delay(0.25)`\
`pulse(1, a90H, 0, d90H)` \
`delay(0.25)`

`pulse(2, a90C, 2, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 1, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 0, d90C)`

`delay(dEvolution)`

`pulse(2, a90C, 1, d45C)`\
`delay(0.25)`\
`pulse(2, a90C, 0, d180C)`\
`delay(0.25)`\
`pulse(2, a90C, 3, d45C)`



In [59]:
def CNOT_operator():
    return hadamard_operator('13C') * CZ_operator() * hadamard_operator('13C')

In [73]:
print(np.array(CNOT_operator()))

[[-0.5-0.5j  0.5-0.5j  0. +0.j   0. +0.j ]
 [ 0.5-0.5j -0.5-0.5j  0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j  -0.5+0.5j -0.5-0.5j]
 [ 0. +0.j   0. +0.j  -0.5-0.5j -0.5+0.5j]]


In [62]:
# rho thermal (not including 1/4 I)
rho = Qobj([[5,0,0,0],
            [0,3,0,0],
            [0,0,-3,0],
            [0,0,0,-5]], dims = [[2,2], [2,2]])

In [63]:
def plot_spectra(rho, gate):
  final = gate * rho * gate.dag()
  final = np.matrix(final)

  a = final[0,0]
  b = final[1,1]
  c = final[2,2]
  d = final[3,3]

  x = ['1H peak 2', '1H peak 1', '13C peak 2', '13C peak 1']
  y = [b-d, a-c, c-d, a-b]

  plt.bar(x, y)
  plt.show()